<a href="https://colab.research.google.com/github/Balaji507/Leaf-disease-detection/blob/main/webpage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 62.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array# load model
import streamlit as st
import cv2


st.set_page_config(page_title='Leaf detector web app')

st.markdown("""
<style> 
    footer {visibility: hidden;}
</style>
""", unsafe_allow_html=True)

st.set_option('deprecation.showfileUploaderEncoding', False)

st.title("""***Leaf Disease detector***""")
st.write("This is a simple image classification web app to predict ")

file = st.file_uploader("Please upload an image png or jpg or jpeg file", type=["png","jpg","jpeg"])


model = tf.keras.models.load_model('/content/drive/MyDrive/Model_Save/classification.h5')


if file is None:
    st.text("You haven't uploaded a png or jpg or jpeg image file")
else:
    state = st.session_state

    def detection_path(classification_result):
      classification_result= classification_result.lower()
      model_path,det_dictionary="",""
      if classification_result=='corn and maize':
       model_path = "/content/drive/MyDrive/Model_Save/corn_and_maize.h5"
       det_dictionary = {0: "blight", 1: "common rust", 2: "gray leaf spot", 3: "healthy"}

      elif classification_result=='cotton':
        model_path = "/content/drive/MyDrive/Model_Save/cotton.h5"
        det_dictionary = {0: "Black root rot", 1: "Lint contamination", 2: "Healthy", 3: "Healthy"}

      elif classification_result=='grape':
        #print('grape')
        model_path = "/content/drive/MyDrive/Model_Save/grape.h5"
        det_dictionary = {0: "Black rot(grape)", 1: "Grape Esca (Black Measles)", 2: "leaf blight (Isariopsis Leaf Spot)", 3: "healthy"}

      elif classification_result=='potato':
        model_path = "/content/drive/MyDrive/Model_Save/potato.h5"
        det_dictionary = {0: "Early blight", 1: "Late blight", 2: "Healthy"}

      elif classification_result=='tomato':
        model_path = "/content/drive/MyDrive/Model_Save/tomato.h5"
        det_dictionary = {0: "bacterial spot", 1: "Early Blight", 2: "Healthy", 3: "Late Blight", 4: "Leaf Mold", 5: "Septoria leaf spot", 6: "Spider mites", 7: "Target spot", 8: "Mosaic virus", 9: "Yellow leaf curl virus"}
      return model_path,det_dictionary

    def preprocess_image(img):
        # Convert to grayscale
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apply histogram equalization
        img_eq = cv2.equalizeHist(img_gray)
        # Apply bilateral filter
        img_filtered = cv2.bilateralFilter(img_eq, 9, 75, 75)
        # Apply binary thresholding
        _, img_thresh = cv2.threshold(img_filtered, 150, 255, cv2.THRESH_BINARY)
        # Convert back to color image
        img_processed = cv2.cvtColor(img_thresh, cv2.COLOR_GRAY2RGB)
        return img_processed
    
    
    def import_and_predict(image_data, model):
        img = image_data.resize((32, 32))
        test_image = img_to_array(img)/255
        test_image = np.expand_dims(test_image, axis = 0)
        result = np.argmax(np.round(model.predict(test_image, verbose=0)))
        result_acc = np.round(model.predict(test_image, verbose=0))
        print(result_acc)
        class_dictionary = {0: "Corn and Maize", 1: "Cotton", 2: "Grape", 3: "Potato", 4: "Tomato"}
        classification_result1 = class_dictionary[result]
        print(classification_result1)
        model_path,det_dictionary=detection_path(classification_result1)
      
        test_image = img_to_array(image_data)/255
        test_image = np.expand_dims(test_image, axis = 0)
        detection_model = tf.keras.models.load_model(model_path)
        disease_result = np.argmax(np.round(detection_model.predict(test_image,verbose=0)))
        detection_result = det_dictionary[disease_result]
        return classification_result1,detection_result,result_acc,disease_result


    image_file = load_img(file,target_size=(256,256))

    prediction = import_and_predict(image_file, model)
    styled_text = f"<h2 style='text-align: center; color: orange;'>Classified Image type: {prediction[0]}</h2>"
    st.write(styled_text, unsafe_allow_html=True)

    styled_text = f"<h2 style='text-align: center; color: green;'>Classified Disease type: {prediction[1]}</h2>"
    st.write(styled_text, unsafe_allow_html=True)
    st.balloons()

Writing app.py


In [ ]:
!streamlit run app.py &npx localtunnel --port 8501 --subdomain leafdetector

npx: installed 22 in 4.72s
your url is: https://leafdetector.loca.lt



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.82.209.81:8501

2023-04-21 07:35:37.740322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 07:35:39.859488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[[0. 0. 0. 1. 0.]]
Potato
